In [1]:
%load_ext autoreload

In [13]:
import src.control.openinterface.commands as commands
import src.control.openinterface.query as query
import src.control.openinterface.packets as packets
import serial

%autoreload
%aimport -serial

In [21]:
ser = serial.Serial('/dev/ttyUSB0', 115200, timeout=0)

def block_read(ser, num):
    reading = bytes([])
    while len(reading) < num:
        reading += ser.read(num)
    return reading

print(ser.name)

/dev/ttyUSB0


In [4]:
for i in range(10):
    ser.write(commands.start())

In [23]:
song = [ (64, 64), (62, 64), (60, 64), (62,64), (64,64), (64,64), 
        (64, 128), (62, 64), (62, 64), (62, 128), (64, 64), (64, 64),
        (64, 128) ]

print(len(song))

cmd = commands.song(0, song)
print(cmd)

13
b'\x8c\x00\r@@>@<@>@@@@@@\x80>@>@>\x80@@@@@\x80'


In [33]:
song = [(64, 32), (64, 32), (65, 32), (67, 32), (67, 32), (65, 32), 
(64, 32), (62, 32), (60, 32), (60, 32), (62, 32), (64, 32),
(64, 48), (62, 16), (62, 255)]

print(len(song))

cmd = commands.song(0, song)
print(cmd)

15
b'\x8c\x00\x0f@ @ A C C A @ > < < > @ @0>\x10>\xff'


In [49]:
song = [(49, 16), (50, 32), (30, 64), (49, 16), (50, 32), (30, 64),
(49, 16), (50, 16), (49, 16), (50, 16), (49, 16), (50, 16), 
(49, 16), (50, 16), (49, 16), (50, 16)]

print(len(song))

cmd = commands.song(2, song)
print(cmd)

16
b'\x8c\x02\x101\x102 \x1e@1\x102 \x1e@1\x102\x101\x102\x101\x102\x101\x102\x101\x102\x10'


In [8]:
song0 = [(76, 16), (75, 16), (76, 16), (75, 16), (76, 16), (71, 16), 
(74, 16), (72, 16), (69, 16), (52, 16), (57, 16), (60, 16), 
(64, 16), (69, 16), (71, 16), (52, 16)]

print(len(song0))

cmd = commands.song(0, song0)
ser.write(cmd)

16


35

In [7]:
song1 =  [(56, 16), (64, 16), (68, 16), (71, 16), (72, 16), (52, 16), 
(57, 16), (64, 16), (76, 16), (75, 16), (76, 16), (75, 16), 
(76, 16), (71, 16), (74, 16), (72, 16)]
print(len(song1))

cmd = commands.song(1, song1)
ser.write(cmd)

16


35

In [9]:
song2 =[(69, 16), (52, 16), (57, 16), (60, 16), (64, 16), (69, 16), 
(71, 16), (52, 16), (56, 16), (64, 16), (72, 16), (71, 16),
(69, 64)] 


print(len(song2))

cmd = commands.song(2, song2)
ser.write(cmd)

13


29

In [14]:
import time
songs = [song0, song1, song2]
for i, song in enumerate(songs):
    duration = sum(map(lambda x: x[1], song))
    duration += 1
    duration /= 64.0
    print("S{}".format(i))
    ser.write(commands.play(i%4))
    time.sleep(duration)
    ser.write(commands.song(i%4, song))

S0
S1
S2


In [5]:
ser.write(commands.mode_full())

1

In [22]:
ser.write(query.sensors(7))
resp = block_read(ser, 1)
s = packets.BumpAndWheelDrop(resp)
print(s.uint)

2


In [82]:
ser.write(commands.drive(0,0))

5

In [75]:
ser.write(commands.motors(False, False, False, False, False))

2

In [17]:
bytes([255])[0]

255

In [ ]:
import threading
%aimport -threading

def readworker(packets):
    packets = [44, 43, 19, 20, 58, 46, 47, 48, 49, 50, 51, 45]
    ser.write(query.stream(packets))
    # Align the data:
    read_data = bytes([])
    aligned = False
    while not aligned:
        read_data = ser.read(32)
        
        